## **YAP 101 Bonus Homework**  
Due Date: 07 April 2020 23:59

Send your files to m.torusdag@etu.edu.tr and m.kutlu@etu.edu.tr

The subject of your email should be "YAP101 HW Bonus"

Total score is 100. This homework is a bonus.

In this homework, you will use sentiment.csv file which is uploaded on Piazza. Each row represents a review about a movie and whether that review is positive (p) or negative (n) or objective (o). 

In [1]:
# Run this cell first
from datascience import * 
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import pandas as pd
import numpy as np

**Question 1 (5 Points)**. Load the data. If there are labels other than p, n, or o, remove those rows. 

In [7]:
# write your answer here
data = pd.read_csv('sentiment.csv')
data['sentiment']=data['sentiment'].str.strip()
data=data.dropna(how='any',axis=0)
data_table=Table.from_df(data)
data_table=data_table.where('sentiment',are.not_equal_to('?'))
data_table.group('sentiment')

sentiment,count
n,78
o,323
p,580


In [8]:
data_table

yorum,sentiment
ben bu filmi seyrettigimde çok etkilenmiştim.,p
Acaba Kate Winslet yerine Drew Barrymore daha mı başa ...,o
Bir kere Leonardo DiCaprio mükemmel oynamış.,p
val kilmer'ı bir çpk insanın sevme nedeni bu filmdir,p
11 oskar ödülüne gelince akademi hiç mi yanlış karar ...,o
'' Aşk filmi''.,o
Bu adamın 2 filmi var ( Kurtlarla Dansı izlemedim) 1.R ...,o
Kahinin sözlerine dikkat edin.,o
Sinemanın zevkine varıyorsunuz izlerken.,p
:) İlk izlediğimde çok etkilenmiştim.,p


**Question 2 (15 Points)** Build a model which predicts the sentiment of each review randomly. Calculate its average accuracy using 5-fold cross validation.  We want to apply 5-fold cross validation without shuffling the data. That is, the first fold should be the first 20% of the data. The second fold should be the second 20% of the data. **DO NOT SHUFFLE THE DATA**. 

In [9]:
numeric_scores = Table().with_columns('sentiment',make_array('p','n','o'),'label',make_array(1,-1,0) )
numeric_scores

sentiment,label
p,1
n,-1
o,0


In [11]:
data2 = data_table.join('sentiment',numeric_scores)
data2

sentiment,yorum,label
n,Ama sinema tarihinin çoğu klasiğini izlemiş bir kimse g ...,-1
n,Benim için şöyle rastgele açılmış bir Yeşilçam filminde ...,-1
n,Böyle kaliteli oyuncuların bu tür basit yapımlarda oyna ...,-1
n,izlenebilir mi ?,-1
n,İzlerken sıkılmıyorsunuz ama salondan cıktıgımda '' off ...,-1
n,Jennifer Lopez eh işte diyorum.,-1
n,Kısaca Titanik bana göre heyecanlı olmayan bir macera f ...,-1
n,kotu bir film.,-1
n,leonardo di caprio olmasaydı ergen kız filmi bile olamazdı.,-1
n,Lopez olmasa 10 olacaktı.,-1


In [18]:
import random
random_score=numeric_scores.column('label')
prediction=[]
for i in np.arange(data_table.num_rows):
    prediction.append(random.choice(random_score))
    
data2=data2.with_column('random_predictions',prediction)   
data2

sentiment,yorum,label,random_predictions
n,Ama sinema tarihinin çoğu klasiğini izlemiş bir kimse g ...,-1,-1
n,Benim için şöyle rastgele açılmış bir Yeşilçam filminde ...,-1,1
n,Böyle kaliteli oyuncuların bu tür basit yapımlarda oyna ...,-1,1
n,izlenebilir mi ?,-1,-1
n,İzlerken sıkılmıyorsunuz ama salondan cıktıgımda '' off ...,-1,1
n,Jennifer Lopez eh işte diyorum.,-1,1
n,Kısaca Titanik bana göre heyecanlı olmayan bir macera f ...,-1,-1
n,kotu bir film.,-1,0
n,leonardo di caprio olmasaydı ergen kız filmi bile olamazdı.,-1,1
n,Lopez olmasa 10 olacaktı.,-1,1


In [23]:
def euclidian_distance(x, y):
    """Returns the Euclidean distance between point1 and point2.
    Each argument is an array containing the coordinates of a point."""
    return np.sqrt(np.sum((x - y)**2))

def all_distances(training, new_point):
    """Returns an array of distances
    between each point in the training set
    and the new point (which is a row of attributes)"""
    attributes = training.select("random_predictions")
    def distance_from_point(row):
        return euclidian_distance(np.array(new_point), np.array(row))
    return attributes.apply(distance_from_point)

def table_with_distances(training, new_point):
    """Augments the training table 
    with a column of distances from new_point"""
    return training.with_column('Distance', all_distances(training, new_point))

def closest(training, new_point, k):
    """Returns a table of the k rows of the augmented table
    corresponding to the k smallest distances"""
    with_dists = table_with_distances(training, new_point)
    sorted_by_distance = with_dists.sort('Distance')
    topk = sorted_by_distance.take(np.arange(k))
    return topk

def majority(topkclasses):
    ones = topkclasses.where('label', are.equal_to(1)).num_rows
    zeros = topkclasses.where('label', are.equal_to(0)).num_rows
    negatives=topkclasses.where('label', are.equal_to(-1)).num_rows
    if ones > zeros and ones>negatives:
        return 1
    elif negatives>ones and negatives>zeros:
        return -1
    else:
        return 0
    
def classify(training, new_point, k):
    closestk = closest(training, new_point, k)
    topkclasses = closestk.select('label')
    return majority(topkclasses)

def count_zero(array):
    return len(array) - np.count_nonzero(array)

def count_equal(array1, array2):
    return count_zero(array1 - array2)

def evaluate_accuracy(training, test, k):
    test_attributes = test.select("random_predictions")
    def classify_testrow(row):
        return classify(training, row, k)
    c = test_attributes.apply(classify_testrow)
    return count_equal(c, test.column('label')) / test.num_rows

def kfold(data,k1,k2):
    index = np.arange(data.num_rows)
    test_size = int(data.num_rows/k1)
    print (test_size)
    total_accuracy = 0
    for i in np.arange(k1):
        test_index = index[i*test_size:(i+1)*test_size]
        train_index = np.setdiff1d(index, test_index)
        test = data.take(test_index)
        train = data.take(train_index)
        accuracy = evaluate_accuracy(train, test, k2)
        total_accuracy = total_accuracy + accuracy
        print (accuracy)
        return total_accuracy/k1

In [25]:
kfold(data2.select("label","random_predictions"),5,5)

196
0.6020408163265306


0.12040816326530612

**Question 3 (80 Points)** Build a model that predicts the sentiment of the reviews. You should define at least 10 features. Try to find the best machine learning model. Try to find the best feature set for this model. In the **same** 5-fold setting, if your model's average accuracy is 0.15 more than random model's performance, you will get full score from this question (still you should define 10 features.) **Show all your work for feature selection etc. including the models that could not perform well.**

In [32]:
data2.column('yorum')[0]

' Ama sinema tarihinin çoğu klasiğini izlemiş bir kimse gelip  Titanik 10 numara  dünyanın en iyi filmidir derse  bu beni derbeder yapar  yollara düşerim.'

In [34]:
def convert(lst): 
    return ([i for item in lst for i in item.split()]) 

In [37]:
res=convert(data2.column('yorum')[0].split()
print(str(res))

SyntaxError: invalid syntax (<ipython-input-37-7265e553e1e5>, line 2)